In [1]:
print("okay")

okay


# **********************************************************************

### Import

In [2]:
# Loading Dependencies
import os
from tqdm import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from keras.layers import Dense, Input, Dropout
from keras.optimizers import Adam
from keras.optimizers import AdamW
from keras.optimizers import Nadam
from keras.optimizers import RMSprop
from keras.models import Model
from keras.callbacks import ModelCheckpoint
#from kaggle_datasets import KaggleDatasets
import transformers
import tokenizers

from tokenizers import BertWordPieceTokenizer

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline


In [ ]:
dir_path = "../"
data_path = "../data"
output_path = "../outputs"

Fonction d'evaluation

In [3]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''

    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

### Preprocessing

For this version of Notebook we will be using TPU's as we have to built a BERT Model

Télécharger les données

In [4]:
# LOADING THE DATA
data_dir_path = data_path+'/jigsaw-toxic'
# dataframe
data = pd.read_csv(data_dir_path+"/jigsaw-toxic-comment-train.csv")
#valid = pd.read_csv(data_dir_path+'/validation.csv')
#test = pd.read_csv(data_dir_path+'/test.csv')
#sub = pd.read_csv(data_dir_path+'/sample_submission.csv')

In [4]:
print("train data : ", data.shape)

train data :  (223549, 8)


truncate data, to reduce the size

In [5]:
#data = data.loc[:10000, :]
#data.shape

In [6]:
#toxic_value_counts = data['toxic'].value_counts()
#toxic_value_counts

### Equilibrer les données
Les données sont de balancé il y plus d'exemple sain que de toxic cela pourrait potentielement biaiser le resultat.

In [5]:
# Séparer les exemples de la classe majoritaire (toxic=0) et de la classe minoritaire (toxic=1)
data_majority = data[data['toxic'] == 0]
data_minority = data[data['toxic'] == 1]

# Sous-échantillonnage des exemples de la classe majoritaire pour équilibrer les classes
data_majority_downsampled = data_majority.sample(n=len(data_minority), random_state=42)

# Concaténer les données sous-échantillonnées de la classe majoritaire avec les données de la classe minoritaire
balanced_data = pd.concat([data_majority_downsampled, data_minority])

# Mélanger les données pour mélanger les classes
balanced_data = balanced_data.sample(frac=1, random_state=42)


#### Afficher la nouvelle répartition des classes

In [6]:
data = balanced_data
print("train data : ", data.shape)

train data :  (42768, 8)


In [9]:
#data = data.loc[:10000, :]
#data.shape

In [7]:
#X_data = data.comment_text.astype(str)
#y_data = data.toxic.values

X_data = data['comment_text'].values
target = data[['toxic', 'insult']].values


Fractionnement des l'ensemble des données : (70% Entrainement, 10% Validation, 20% Testes) | Data Splitting (70% Training, 10% Validation, 20% Testing)

In [8]:
d_train, d_test, t_train, t_test = model_selection.train_test_split(X_data, target, test_size=0.2, random_state=42) # Test : 20%
d_train, d_valid, t_train, t_valid = model_selection.train_test_split(d_train, t_train, test_size=0.125, random_state=1) # Validation : 10%

In [9]:
print("train data : ", d_train.shape)
print("valid data : ", d_valid.shape)
print("test data : ", d_test.shape)

train data :  (29937,)
valid data :  (4277,)
test data :  (8554,)


In [10]:
print("train target : ", t_train.shape)
print("valid target : ", t_valid.shape)
print("test target : ", t_test.shape)

train target :  (29937, 2)
valid target :  (4277, 2)
test target :  (8554, 2)


Encoder For DATA for understanding what encode batch does read documentation of hugging face tokenizer :

In [11]:
def text_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    """
    Encoder for encoding the text into sequence of integers for BERT Input
    """
    # Set maximum length
    tokenizer.enable_truncation(max_length=maxlen)

    #tokenizer.enable_padding(max_length=maxlen)
    # Enable padding
    #tokenizer.enable_padding(pad_id=0, pad_token="[PAD]", pad_to_multiple_of=None)
    tokenizer.enable_padding()
    all_ids = []

    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])

    return np.array(all_ids)

## Tokenization

For understanding please refer to hugging face documentation again

In [12]:
# First load the real tokenizer
try:
    tokenizer = transformers.AutoTokenizer.from_pretrained(output_path+'/tokenizers')
except (OSError, ValueError):
    tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
    # Save the loaded tokenizer locally
    tokenizer.save_pretrained(output_path+'/tokenizers')


In [13]:
#IMP DATA FOR CONFIG

# Configuration
EPOCHS = 15
BATCH_SIZE = 32 #* strategy.num_replicas_in_sync
MAX_LEN = 192


WordPiece is the tokenization algorithm Google developed to pretrain BERT. It has since been reused in quite a few Transformer models based on BERT, such as DistilBERT, MobileBERT, Funnel Transformers, and MPNET. It’s very similar to BPE in terms of the training, but the actual tokenization is done differently.

In [14]:
text_tokenizer = BertWordPieceTokenizer(output_path+'/tokenizers/vocab.txt', lowercase=False)
text_tokenizer

Tokenizer(vocabulary_size=119547, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=False, wordpieces_prefix=##)

In [18]:
x_train = text_encode(d_train, text_tokenizer, maxlen=MAX_LEN)
x_valid = text_encode(d_valid, text_tokenizer, maxlen=MAX_LEN)
x_test = text_encode(d_test, text_tokenizer, maxlen=MAX_LEN)

y_train = t_train[:,0]
y_valid = t_valid[:,0]
y_test = t_test[:,0]

100%|██████████| 34/34 [00:01<00:00, 25.53it/s]


In [19]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(tf.data.experimental.AUTOTUNE)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

## Création du modèle.

Dans la ligne `sequence_output = transformer(input_word_ids)[0]`, `transformer` fait référence à l'objet `transformer_layer` qui est passé à la fonction `build_bert_model` en tant que paramètre. Cet objet est une instance du modèle BERT (ou DistilBERT dans ce cas) chargé à partir de TensorFlow Hub ou d'une autre source pré-entraînée.

Expliquons la ligne en détail :

1. **`transformer(input_word_ids)`** :
   - `transformer` est l'objet représentant le modèle BERT chargé.
   - `input_word_ids` est l'entrée du modèle, qui contient les identifiants de tokens des mots du texte à encoder.

2. **`[0]`** :
   - Après l'appel de `transformer(input_word_ids)`, le modèle BERT renvoie plusieurs sorties. Pour récupérer la sortie principale, qui est généralement la séquence d'embeddings de chaque token dans le texte d'entrée, on sélectionne l'élément `[0]` de la sortie.
   - Dans le cas de BERT, cette sortie est généralement une paire de tenseurs, où le premier élément représente la séquence d'embeddings de chaque token, et le deuxième élément représente un tenseur contenant diverses informations supplémentaires telles que l'état caché du modèle.

Ainsi, `sequence_output` contient la séquence d'embeddings de chaque token dans le texte d'entrée, qui peut être utilisée pour effectuer des tâches de classification, de régression ou d'autres tâches de traitement du langage naturel.

In [20]:
from transformers import TFDistilBertModel, DistilBertTokenizer
import tensorflow as tf
from keras import regularizers
def build_bert_model(max_len=512, optimizer = Adam()):
  """
  That function create the BERT model for training
  """
  # Charger le modèle pré-entraîné DistilBERT et le tokenizer
  distilbert_model = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
  #tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

  model = tf.keras.Sequential([
    # La couche d'entrée
    Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids"),

    # Ajouter la couche DistilBERT (notez que nous utilisons distilbert_model.layers[0] pour accéder à la couche de transformer)
    # La couche DistilBERT
    distilbert_model.layers[0],

    # La couche pour obtenir le premier token [CLS]
    tf.keras.layers.Lambda(lambda seq: seq[:, 0, :]),

    #Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    #Dropout(0.5),
    #Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    #Dropout(0.5),
    #Dense(64,  activation="relu", kernel_regularizer=regularizers.l2(0.01)),

    # Ajouter des couches supplémentaires si nécessaire
    # Par exemple, une couche Dense pour la classification
    # La couche de sortie
    Dense(1, activation='sigmoid')
  ])

  loss = tf.keras.losses.BinaryCrossentropy()
  #metrics = tf.metrics.BinaryAccuracy()

  # Compiler le modèle
  # Compiler le modèle avec une loss adaptée à la classification binaire
  model.compile(optimizer = optimizer, loss=loss, metrics=['accuracy'])

  # Afficher le résumé du modèle

  return model

In [ ]:
from transformers import TFDistilBertModel
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def build_bert_model(max_len=192, optimizer=Adam(learning_rate=1e-5), dropout_rate=0.2):
    """
    Cette fonction crée le modèle BERT pour l'entraînement
    """
    # Charger le modèle pré-entraîné DistilBERT
    distilbert_model = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

    # Créer un modèle séquentiel
    model = tf.keras.Sequential([
        # La couche d'entrée
        Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids"),

        # Ajouter la couche DistilBERT
        distilbert_model,

        # La couche pour obtenir le premier token [CLS]
        Lambda(lambda seq: seq[:, 0, :]),

        # Ajouter des couches supplémentaires
        # Dense(256, activation='relu'),
        # Dropout(dropout_rate),
        # Dense(128, activation='relu'),
        # Dropout(dropout_rate),

        # La couche de sortie
        Dense(1, activation='sigmoid')
    ])

    # Définir la fonction de perte et compiler le modèle
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.AUC()])

    return model

# Paramètres du modèle
MAX_LEN = 192
EPOCHS = 15
BATCH_SIZE = 32

# Créer et compiler le modèle
model = build_bert_model(max_len=MAX_LEN)

# Afficher le résumé du modèle
model.summary()


In [21]:

model = build_bert_model(optimizer = Adam(learning_rate = 1e-5 ), max_len=MAX_LEN)
model.load_weights(output_path+'/bert_model-3.h5') 
#model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [34]:
new_model = build_bert_model()  # Créez le modèle avec la même architecture
new_model.load_weights(output_path+'/bert_model-5-val.h5')  # Chargez les poids sauvegardés

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


### Evaluate

If you want to use any another model just replace the model name in transformers._____ and use accordingly

In [35]:
test_loss, test_accuracy = model.evaluate(x_test,y_test)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

268/268 [==============================] - 864s 3s/step - loss: 0.4636 - accuracy: 0.9183
Test Loss: 0.46361732482910156
Test Accuracy: 0.9182838201522827


In [23]:
loss, accuracy = model.evaluate(valid_dataset)

print(f'Valid Loss: {loss}')
print(f' Valid Accuracy: {accuracy}')



134/134 [==============================] - 472s 3s/step - loss: 0.4932 - accuracy: 0.9133
Loss: 0.49315693974494934
Accuracy: 0.9132569432258606


In [24]:
pred = model.predict(test_dataset, verbose=1)


268/268 [==============================] - 912s 3s/step


In [25]:
print("Auc: %.2f%%" % (roc_auc(pred,y_test)))

Auc: 0.97%


In [28]:
y_valid_pred = pred

In [30]:
from sklearn.metrics import roc_auc_score, f1_score

# Après l'entraînement du modèle, tu peux évaluer les performances sur l'ensemble de validation ou de test
# par exemple, si tu as déjà un modèle entraîné `model`

# Prédictions sur l'ensemble de validation
#y_valid_pred = model.predict(valid_dataset)

# Calcul du score ROC-AUC
roc_auc = roc_auc_score(y_test, pred)

# Calcul du score F1
# Il faut convertir les prédictions en classes binaires (0 ou 1) en utilisant un seuil (par exemple, 0.5)
y_pred_binary = (pred > 0.5).astype(int)
f1 = f1_score(y_test, y_pred_binary)

print("ROC-AUC score:", roc_auc)
print("F1 score:", f1)


ROC-AUC score: 0.9740666451818
F1 score: 0.9207932011331444
